In [1]:
import pandas as pd

## Get daily dialog data

In [2]:
# Get daily dialog data

with open('datasets/data_raw/dailydialog/dialogues_text.txt', encoding= 'utf-8') as f:
    dialogue = f.readlines()

with open('datasets/data_raw/dailydialog/dialogues_emotion.txt', encoding= 'utf-8') as f:
    emotion = f.readlines()

d=[]
sentences = []
for line in dialogue:
    line = line.replace('\n', '')
    d.append(line.split('__eou__'))

for lis in d:
    for sentence in lis:
        sentences.append(sentence)

sentences = list(filter(None,sentences))

emotion = ' '.join(emotion).split()

df = pd.DataFrame(zip(emotion, sentences), columns=['emotion','text'])
df.to_csv('datasets/dailydialog.csv')
df.head()


,emotion,text
0,2,The kitchen stinks .
1,0,I'll throw out the garbage .
2,4,"So Dick , how about getting some coffee for to..."
3,2,Coffee ? I don ’ t honestly like that kind of...
4,0,"Come on , you can at least try a little , bes..."


## Get emotion stimulus data

In [3]:
# Get emotion stimulus data

with open('datasets/data_raw/emotion_stimulus/Emotion Cause.txt', encoding= 'utf-8') as f:
    emotion_stimulus = f.readlines()

with open('datasets/data_raw/emotion_stimulus/No Cause.txt', encoding= 'utf-8') as f:
    emotion_stimulus_no_cause = f.readlines()

emotion_stimulus_all = emotion_stimulus+emotion_stimulus_no_cause
print(len(emotion_stimulus_all))

2414


In [4]:
df_emotion_stimulus = pd.DataFrame(emotion_stimulus_all,columns=['text'])

df_emotion_stimulus['emotion'] = df_emotion_stimulus['text'].apply(lambda x: str(x.split('>')[0]))

In [5]:
df_emotion_stimulus.head()

,text,emotion
0,<happy>I suppose I am happy <cause>being so ` ...,<happy
1,<happy>Lennox has always truly wanted to fight...,<happy
2,"<happy>He was a professional musician now , st...",<happy
3,<happy>Holmes is happy <cause>having the freed...,<happy
4,<happy>I had problems with tutors trying to en...,<happy


In [6]:
def clean_text(text):
    import re
    rep = {'<cause>': '',
           r'<\\cause>': '',
           '<happy>': '',
           '<sad>': '',
           '<anger>': '',
           '<fear>': '',
           '<suprise>': '',
           '<shame>': '',
           '<disgust>': ''}

    for item in rep.keys():
        text = re.sub(item, rep[item], text)
        
    return text


df_emotion_stimulus['text'] =  df_emotion_stimulus['text'].apply(lambda x: clean_text(x))

df_emotion_stimulus['emotion'] = df_emotion_stimulus['emotion'].apply(lambda x: x.replace('<',''))
df_emotion_stimulus.to_csv('datasets/emotion_stimulus.csv')
df_emotion_stimulus= df_emotion_stimulus[['emotion','text']]

df_emotion_stimulus.head()

,emotion,text
0,happy,I suppose I am happy being so ` tiny' ; it mea...
1,happy,Lennox has always truly wanted to fight for th...
2,happy,"He was a professional musician now , still sen..."
3,happy,Holmes is happy having the freedom of the hous...
4,happy,I had problems with tutors trying to encourage...


## Get Isear data

In [7]:
df_isear = pd.read_csv('datasets/data_raw/isear.csv', sep='|',header=None, names=['ID','CITY','COUN','SUBJ','SEX','AGE','RELI','PRAC','FOCC','MOCC','FIEL','EMOT','WHEN','LONG','INTS','ERGO','TROPHO','TEMPER','EXPRES','MOVE','EXP1','EXP2','EXP10','PARAL','CON','EXPC','PLEA','PLAN','FAIR','CAUS','COPING','MORL','SELF','RELA','VERBAL','NEUTRO','Field1','Field3','Field2','MYKEY','SIT','STATE'])

df_isear.columns = ['CITY','COUN','SUBJ','SEX','AGE','RELI','PRAC','FOCC','MOCC','FIEL','EMOT','WHEN','LONG','INTS','ERGO','TROPHO','TEMPER','EXPRES','MOVE','EXP1','EXP2','EXP10','PARAL','CON','EXPC','PLEA','PLAN','FAIR','CAUS','COPING','MORL','SELF','RELA','VERBAL','NEUTRO','Field1','Field3','Field2','MYKEY','SIT','STATE','']

df_isear = df_isear[['Field1','SIT']]
df_isear = df_isear[df_isear.Field1!='Field1']
df_isear.reset_index(drop=True, inplace=True)
df_isear.columns=['emotion','text']
df_isear.to_csv('datasets/isear.csv')
df_isear.head()

,emotion,text
0,joy,"During the period of falling in love, each tim..."
1,fear,When I was involved in a traffic accident.
2,anger,When I was driving home after several days of...
3,sadness,When I lost the person who meant the most to me.
4,disgust,The time I knocked a deer down - the sight of ...


In [9]:
frames = [df_isear, df_emotion_stimulus,df]
df = pd.concat(frames)

In [11]:
df['emotion'].value_counts()

0           85572
4           12885
6            1823
anger        1579
fear         1518
shame        1242
disgust      1191
5            1150
sadness      1096
joy          1094
guilt        1093
1            1022
sad           575
happy         479
2             353
surprise      213
3             174
Name: emotion, dtype: int64

In [15]:

def clean_text(text):
    import re
    replace_emotions = {'0': 'neutral',
                    '1': 'anger',
                    '2': 'disgust',
                    '3': 'fear',
                    '4': 'happy',
                    '5': 'sad',
                    '6': 'suprise',
                    'sadness': 'sad'}

    for item in replace_emotions.keys():
        text = re.sub(item, replace_emotions[item], text)
        
    return text

df['emotion'] =  df['emotion'].apply(lambda x: clean_text(x))
df.to_csv('datasets/dataset_complete.csv')

In [14]:
df['emotion'].value_counts()

neutral     85572
happy       13364
sad          2821
anger        2601
suprise      1823
fear         1692
disgust      1544
shame        1242
joy          1094
guilt        1093
surprise      213
Name: emotion, dtype: int64